In [2]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import datetime
import yfinance

In [7]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=2)

In [12]:
btc_price = vbt.YFData.download(
    'BTC-USD',
    interval='1m',
    missing_index=True,
    start=start_date.timestamp(),
    end=end_date.timestamp()
    ).get('Close')

In [16]:
btc_price,range_indexes = btc_price.vbt.range_split(n=100,range_len=1440)

In [46]:
def optimize_rsi(close,window,entry,exit):
    rsi = vbt.IndicatorFactory.from_talib('RSI').run(close, timeperiod=window).real
    return rsi<entry, rsi>exit

In [47]:
rsi_ind = vbt.IndicatorFactory(
            class_name='OptimizeRSI',
            short_name='rsi',
            input_names=['close'],
            param_names=['window', 'entry', 'exit'],
            output_names=['entries','exits']
        ).from_apply_func(
                optimize_rsi,
                window=14,
                entry=30,
                exit=70)

In [48]:
step_size=10
entries = np.arange(10,45,step=step_size,dtype=int)
exits = np.arange(55,95,step=step_size,dtype=int)
windows = np.arange(10,45,step=step_size,dtype=int)

In [73]:
rsi_res = rsi_ind.run(
        btc_price,
        window=windows,
        entry=entries,
        exit=exits,
        param_product=True)

In [74]:
rsi_entries = rsi_res.entries
rsi_exits = rsi_res.exits

In [75]:
rsi_entries.iloc[-1,:] = True

In [76]:
rsi_pf = vbt.Portfolio.from_signals(
        btc_price,
        rsi_entries,
        rsi_exits,
        freq='1T',
        fees=0.001)

In [77]:
fig = rsi_pf.total_return().vbt.volume(
        x_level='rsi_exit',
        y_level='rsi_entry',
        z_level='rsi_window',
        slider_level='split_idx')

In [78]:
fig.show()

##### HeatMap

In [79]:
rsi_res = rsi_ind.run(
        btc_price,
        window=14, ### changed window=windows to window=14 for 2d graph
        entry=entries,
        exit=exits,
        param_product=True)
rsi_entries = rsi_res.entries
rsi_exits = rsi_res.exits
rsi_entries.iloc[-1,:] = True

rsi_pf = vbt.Portfolio.from_signals(
        btc_price,
        rsi_entries,
        rsi_exits,
        freq='1T',
        fees=0.001)

#### Plotting HeatMap
fig = rsi_pf.total_return().vbt.heatmap(
    x_level='rsi_exit',
    y_level='rsi_entry',
    slider_level='split_idx'
    )
fig.show()

###### the parammeters is not giving any common value on slider

In [82]:
rsi_res = rsi_ind.run(
        btc_price,
        window=14, ### changed window=windows to window=14 for 2d graph
        entry=entries,
        exit=exits,
        param_product=True)
rsi_entries = rsi_res.entries
rsi_exits = rsi_res.exits
rsi_entries.iloc[-1,:] = True

rsi_pf = vbt.Portfolio.from_signals(
        btc_price,
        rsi_entries,
        rsi_exits,
        freq='1T',
        fees=0.001)

# taking average all the windows
rsi_tot_returns = rsi_pf.total_return().groupby(level=['rsi_exit','rsi_entry']).mean()

print(rsi_tot_returns)
# Plotting HeatMap
fig = rsi_tot_returns.vbt.heatmap(
    x_level='rsi_exit',
    y_level='rsi_entry',
#     slider_level='split_idx'
    )
fig.show()

rsi_exit  rsi_entry
55        10          -0.003854
          20          -0.016009
          30          -0.027660
          40          -0.053446
65        10          -0.002482
          20          -0.014598
          30          -0.021842
          40          -0.036931
75        10          -0.000223
          20          -0.009574
          30          -0.012533
          40          -0.020642
85        10           0.002518
          20          -0.005832
          30          -0.001740
          40          -0.004219
Name: total_return, dtype: float64


 ###### All are negative returns or very very small. Therefore, not good to go with

#### BoxPlot

In [87]:
rsi_res = rsi_ind.run(
        btc_price,
        window=14, 
        entry=20, # from previous heatmap
        exit=80,
        param_product=True)
rsi_entries = rsi_res.entries
rsi_exits = rsi_res.exits
rsi_entries.iloc[-1,:] = True

rsi_pf = vbt.Portfolio.from_signals(
        btc_price,
        rsi_entries,
        rsi_exits,
        freq='1T',
        fees=0.001)

# taking average all the windows
rsi_tot_returns = rsi_pf.total_return()

print(rsi_tot_returns)
# Plotting BoxPlot
box = vbt.plotting.Box(
            data=rsi_tot_returns,
            trace_names=['RSI_strat']
            )
box.fig.show()

rsi_window  rsi_entry  rsi_exit  split_idx
14          20         80        0           -0.016124
                                 1           -0.014922
                                 2           -0.015339
                                 3           -0.014841
                                 4           -0.015623
                                                ...   
                                 95          -0.004752
                                 96          -0.005104
                                 97          -0.005679
                                 98          -0.005332
                                 99          -0.004665
Name: total_return, Length: 100, dtype: float64
